# Model Making Kitchen

Create three candidate models for the ensemble modeling challenge

Setting: New York State, Timepoint 2: July 15, 2021

### Gather population values and initial conditions for this setting
Population values from [Census Reporter - New York](https://censusreporter.org/profiles/04000US36-new-york/)

Initial conditions estimated from [COVID-19 Forecast Hub](https://github.com/reichlab/covid19-forecast-hub/tree/master/data-truth)

In [1]:
# Population
NYS_total_pop = 19_677_152
pop_frac = [0.202, 0.617, 0.181]
NYS_young_pop = NYS_total_pop*pop_frac[0] # under 18
NYS_middle_pop = NYS_total_pop*pop_frac[1] # ages 18 - 64
NYS_old_pop = NYS_total_pop*pop_frac[2] # ages 65+

# Initial conditions `{age}_initials = [E0, I0, R0, H0, D0]`
cumul_cases0 = 2_102_869.0
cumul_hosp0 = 136_862.0
all_initials = [4_000.0, 4_000.0, 2_102_000.0, 700.0, 53_123.0]
y_initials = [i * pop_frac[0] for i in all_initials]
m_initials = [i * pop_frac[1] for i in all_initials]
o_initials = [i * pop_frac[2] for i in all_initials]

### Load dependencies

In [2]:
import os
import json
import sympy

from mira.metamodel import *
from mira.examples.concepts import susceptible, exposed, infected, recovered
from mira.modeling import Model
from mira.modeling.amr.petrinet import AMRPetriNetModel, template_model_to_petrinet_json
from mira.sources.amr.petrinet import template_model_from_amr_json
from mira.metamodel.io import model_to_json_file, model_from_json_file

# Model 1: Age-structured SEIRHD model
Adapted from the Scenario 3 model for the 18-month evaluation, uses the same contact matrix

### (1) Define units

In [3]:
person_units = lambda: Unit(expression=sympy.Symbol('person'))
day_units = lambda: Unit(expression=sympy.Symbol('day'))
per_day_units = lambda: Unit(expression=1/sympy.Symbol('day'))
dimensionless_units = lambda: Unit(expression=sympy.Integer('1'))
per_day_per_person_units = lambda: Unit(expression=1/(sympy.Symbol('day')*sympy.Symbol('person')))

### (2) Define and stratify model concepts

In [6]:
_susceptible = Concept(name='S', units=person_units(), identifiers={'ido': '0000514'})
_exposed = Concept(name='E', units=person_units(), identifiers={'apollosv': '00000154'})
_infected = Concept(name='I', units=person_units(), identifiers={'ido': '0000511'})
_recovered = Concept(name='R', units=person_units(), identifiers={'ido': '0000592'})
_hospitalized = Concept(name="H", units=person_units(), identifiers={"ido": "0000511"},
                        context={"property": "ncit:C25179"})
_deceased = Concept(name="D", units=person_units(), identifiers={"ncit": "C28554"})

c = {
    'S_y': _susceptible.with_context(status="young"),
    'S_m': _susceptible.with_context(status="middle"),
    'S_o': _susceptible.with_context(status="old"),
    
    'E_y': _exposed.with_context(status="young"),
    'E_m': _exposed.with_context(status="middle"),
    'E_o': _exposed.with_context(status="old"),
    
    'I_y': _infected.with_context(status="young"),
    'I_m': _infected.with_context(status="middle"),
    'I_o': _infected.with_context(status="old"),
    
    'R_y': _recovered.with_context(status="young"),
    'R_m': _recovered.with_context(status="middle"),
    'R_o': _recovered.with_context(status="old"),
    
    'H_y': _recovered.with_context(status="young"),
    'H_m': _recovered.with_context(status="middle"),
    'H_o': _recovered.with_context(status="old"),
    
    'D_y': _recovered.with_context(status="young"),
    'D_m': _recovered.with_context(status="middle"),
    'D_o': _recovered.with_context(status="old"),
    
    "Cumulative_cases": Concept(name="Cumulative_cases", units=person_units()),
    "Cumulative_hosp": Concept(name="Cumulative_hosp", units=person_units())
}

for concept in c:
    c[concept].name = concept

### (3) Define parameters with uncertainty
Parameter values estimated from [CDC COVID-19 Pandemic Planning Scenarios](https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html)

Contact matrix from [18-month Epi Eval Scenario 3 model](https://github.com/DARPA-ASKEM/program-milestones/blob/main/18-month-milestone/evaluation/Epi%20Use%20Case/ASKEM_18Month_Epi_Evaluation_Scenarios_03.22.2024_FINAL.pdf) 

In [7]:
parameters = {
    'beta': Parameter(name='beta', value=sympy.Float(0.2), units=per_day_units(),
                      distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.05,
                                                            'maximum': 0.8})),  # Transmission rate
    'N': Parameter(name='total_population', value=sympy.Float(NYS_total_pop), units=person_units()),  # Total population
    
    'r_EI': Parameter(name='r_EI', value=sympy.Float(0.16), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.1,
                                                            'maximum': 0.25})),  # Rate of progressing E -> I
    
    'r_IR_y': Parameter(name='r_IR_y', value=sympy.Float(0.475), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.4,
                                                            'maximum': 0.5})),  # Rate of progressing I_y -> R_y
    'r_IR_m': Parameter(name='r_IR_m', value=sympy.Float(0.15), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.1,
                                                        'maximum': 0.2})),  # Rate of progressing I_m -> R_m
    'r_IR_o': Parameter(name='r_IR_o', value=sympy.Float(0.2125), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.15,
                                                        'maximum': 0.25})),  # Rate of progressing I_o -> R_o

    'r_IH_y': Parameter(name='r_IH_y', value=sympy.Float(0.025), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.02,
                                                            'maximum': 0.03})),  # Rate of progressing I_y -> H_y
    'r_IH_m': Parameter(name='r_IH_m', value=sympy.Float(0.016), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.01,
                                                        'maximum': 0.1})),  # Rate of progressing I_m -> H_m
    'r_IH_o': Parameter(name='r_IH_o', value=sympy.Float(0.0375), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.02,
                                                        'maximum': 0.04})),  # Rate of progressing I_o -> H_o
    
    'r_HR_y': Parameter(name='r_HR_y', value=sympy.Float(0.331), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.3,
                                                            'maximum': 0.4})),  # Rate of progressing H_y -> R_y
    'r_HR_m': Parameter(name='r_HR_m', value=sympy.Float(0.1583), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.1,
                                                        'maximum': 0.2})),  # Rate of progressing H_m -> R_m
    'r_HR_o': Parameter(name='r_HR_o', value=sympy.Float(0.116), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.09,
                                                        'maximum': 0.13})),  # Rate of progressing H_o -> R_o
    
    'r_HD_y': Parameter(name='r_HD_y', value=sympy.Float(0.002), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.001,
                                                            'maximum': 0.003})),  # Rate of progressing H_y -> D_y
    'r_HD_m': Parameter(name='r_HD_m', value=sympy.Float(0.0083), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.008,
                                                        'maximum': 0.009})),  # Rate of progressing D_m -> D_m
    'r_HD_o': Parameter(name='r_HD_o', value=sympy.Float(0.0268), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.02,
                                                        'maximum': 0.035})),  # Rate of progressing D_o -> D_o
    
    'Myy': Parameter(name='Myy', value=sympy.Float(38.62), units=per_day_units()),  # Contact rate young -> young
    'Mym': Parameter(name='Mym', value=sympy.Float(20.56), units=per_day_units()),  # Contact rate young -> middle
    'Myo': Parameter(name='Myo', value=sympy.Float(6.12), units=per_day_units()),  # Contact rate young -> old
    'Mmy': Parameter(name='Mmy', value=sympy.Float(20.56), units=per_day_units()),  # Contact rate middle -> young
    'Mmm': Parameter(name='Mmm', value=sympy.Float(28.22), units=per_day_units()),  # Contact rate middle -> middle
    'Mmo': Parameter(name='Mmo', value=sympy.Float(11.6), units=per_day_units()),  # Contact rate middle -> old
    'Moy': Parameter(name='Moy', value=sympy.Float(6.12), units=per_day_units()),  # Contact rate old -> young
    'Mom': Parameter(name='Mom', value=sympy.Float(11.6), units=per_day_units()),  # Contact rate old -> middle
    'Moo': Parameter(name='Moo', value=sympy.Float(20.01), units=per_day_units()),  # Contact rate old -> old
}

### (4) Define `SymPy` variables

In [8]:
S_y, S_m, S_o, E_y, E_m, E_o, I_y, I_m, I_o, R_y, R_m, R_o, H_y, H_m, H_o, D_y, D_m, D_o, beta, N, r_EI, r_IR_y, r_IR_m, r_IR_o, r_IH_y, r_IH_m, r_IH_o, r_HR_y, r_HR_m, r_HR_o, r_HD_y, r_HD_m, r_HD_o, Myy, Mym, Myo, Mmy, Mmm, Mmo, Moy, Mom, Moo = \
    sympy.symbols(
        'S_y S_m S_o E_y E_m E_o I_y I_m I_o R_y R_m R_o H_y H_m H_o D_y D_m D_o beta N r_EI r_IR_y r_IR_m r_IR_o r_IH_y r_IH_m r_IH_o r_HR_y r_HR_m r_HR_o r_HD_y r_HD_m r_HD_o Myy Mym Myo Mmy Mmm Mmo Moy Mom Moo'
    )

### (5) Set initial conditions

In [9]:
initials = {
    "S_y": Initial(concept=c["S_y"], expression=NYS_young_pop - sum(y_initials)),
    "S_m": Initial(concept=c["S_m"], expression=NYS_middle_pop - sum(m_initials)),
    "S_o": Initial(concept=c["S_o"], expression=NYS_old_pop - sum(o_initials)),
    "E_y": Initial(concept=c["E_y"], expression=y_initials[0]),
    "E_m": Initial(concept=c["E_m"], expression=m_initials[0]),
    "E_o": Initial(concept=c["E_o"], expression=o_initials[0]),
    "I_y": Initial(concept=c["I_y"], expression=y_initials[1]),
    "I_m": Initial(concept=c["I_m"], expression=m_initials[1]),
    "I_o": Initial(concept=c["I_o"], expression=o_initials[1]),
    "R_y": Initial(concept=c["R_y"], expression=y_initials[2]),
    "R_m": Initial(concept=c["R_m"], expression=m_initials[2]),
    "R_o": Initial(concept=c["R_o"], expression=o_initials[2]),
    'H_y': Initial(concept=c["H_y"], expression=y_initials[3]),
    'H_m': Initial(concept=c["H_m"], expression=m_initials[3]),
    'H_o': Initial(concept=c["H_o"], expression=o_initials[3]),
    'D_y': Initial(concept=c["D_y"], expression=y_initials[4]),
    'D_m': Initial(concept=c["D_m"], expression=m_initials[4]),
    'D_o': Initial(concept=c["D_o"], expression=o_initials[4]),
    "Cumulative_cases": Initial(concept=c["Cumulative_cases"], expression=cumul_cases0),
    "Cumulative_hosp": Initial(concept=c["Cumulative_hosp"], expression=cumul_hosp0)
}

### (6) Define templates

In [11]:
##### S -> E
# Sy -> Ey by Iy
syeyiy = ControlledConversion(
    subject=c['S_y'],
    outcome=c['E_y'],
    controller=c['I_y'],
    rate_law=beta*S_y*(Myy*I_y) / N
)
# Sy -> Ey by Im
syeyim = ControlledConversion(
    subject=c['S_y'],
    outcome=c['E_y'],
    controller=c['I_m'],
    rate_law=beta*S_y*(Mym*I_m) / N
)
# Sy -> Ey by Io
syeyio = ControlledConversion(
    subject=c['S_y'],
    outcome=c['E_y'],
    controller=c['I_o'],
    rate_law=beta*S_y*(Myo*I_o) / N
)

# Sm -> Em by Iy
smemiy = ControlledConversion(
    subject=c['S_m'],
    outcome=c['E_m'],
    controller=c['I_y'],
    rate_law=beta*S_m*(Mmy*I_y) / N
)
# Sm -> Em by Im
smemim = ControlledConversion(
    subject=c['S_m'],
    outcome=c['E_m'],
    controller=c['I_m'],
    rate_law=beta*S_m*(Mmm*I_m) / N
)
# Sm -> Em by Io
smemio = ControlledConversion(
    subject=c['S_m'],
    outcome=c['E_m'],
    controller=c['I_o'],
    rate_law=beta*S_m*(Mmo*I_o) / N
)

# So -> Eo by Iy
soeoiy = ControlledConversion(
    subject=c['S_o'],
    outcome=c['E_o'],
    controller=c['I_y'],
    rate_law=beta*S_o*(Moy*I_y) / N
)
# So -> Eo by Im
soeoim = ControlledConversion(
    subject=c['S_o'],
    outcome=c['E_o'],
    controller=c['I_m'],
    rate_law=beta*S_o*(Mom*I_m) / N
)
# So -> Eo by Io
soeoio = ControlledConversion(
    subject=c['S_o'],
    outcome=c['E_o'],
    controller=c['I_o'],
    rate_law=beta*S_o*(Moo*I_o) / N
)


#### E -> I
# Ey -> Iy
eyiy = NaturalConversion(
    subject=c['E_y'],
    outcome=c['I_y'],
    rate_law=r_EI*E_y
)
# Em -> Im
emim = NaturalConversion(
    subject=c['E_m'],
    outcome=c['I_m'],
    rate_law=r_EI*E_m
)
# Eo -> Io
eoio = NaturalConversion(
    subject=c['E_o'],
    outcome=c['I_o'],
    rate_law=r_EI*E_o
)


#### I -> R
# Iy -> Ry
iyry = NaturalConversion(
    subject=c['I_y'],
    outcome=c['R_y'],
    rate_law=r_IR_y*I_y
)
# Im -> Rm
imrm = NaturalConversion(
    subject=c['I_m'],
    outcome=c['R_m'],
    rate_law=r_IR_m*I_m
)
# Io -> Ro
ioro = NaturalConversion(
    subject=c['I_o'],
    outcome=c['R_o'],
    rate_law=r_IR_m*I_o
)


#### I -> H
# Iy -> Hy
iyhy = NaturalConversion(
    subject=c['I_y'],
    outcome=c['H_y'],
    rate_law=r_IH_y*I_y
)
# Im -> Hm
imhm = NaturalConversion(
    subject=c['I_m'],
    outcome=c['H_m'],
    rate_law=r_IH_m*I_m
)
# Io -> Ho
ioho = NaturalConversion(
    subject=c['I_o'],
    outcome=c['H_o'],
    rate_law=r_IH_o*I_o
)


#### H -> R
# Hy -> Ry
hyry = NaturalConversion(
    subject=c['H_y'],
    outcome=c['R_y'],
    rate_law=r_HR_y*H_y
)
# Hm -> Rm
hmrm = NaturalConversion(
    subject=c['H_m'],
    outcome=c['R_m'],
    rate_law=r_HR_m*H_m
)
# Ho -> Ro
horo = NaturalConversion(
    subject=c['H_o'],
    outcome=c['R_o'],
    rate_law=r_HR_o*H_o
)


#### H -> D
# Hy -> Dy
hydy = NaturalConversion(
    subject=c['H_y'],
    outcome=c['D_y'],
    rate_law=r_HD_y*H_y
)
# Hm -> Dm
hmdm = NaturalConversion(
    subject=c['H_m'],
    outcome=c['D_m'],
    rate_law=r_HD_m*H_m
)
# Ho -> Do
hodo = NaturalConversion(
    subject=c['H_o'],
    outcome=c['D_o'],
    rate_law=r_HD_o*H_o
)


### Cumulative Cases
# Ey -> Iy
ccy = ControlledProduction(
    controller=c['E_y'],
    outcome=c['Cumulative_cases'],
    rate_law=r_EI*E_y
)
# Em -> Im
ccm = ControlledProduction(
    controller=c['E_m'],
    outcome=c['Cumulative_cases'],
    rate_law=r_EI*E_m
)
# Eo -> Io
cco = ControlledProduction(
    controller=c['E_o'],
    outcome=c['Cumulative_cases'],
    rate_law=r_EI*E_o
)


### Cumulative Hospitalizations
# Iy -> Hy
chy = NaturalConversion(
    subject=c['I_y'],
    outcome=c['Cumulative_hosp'],
    rate_law=r_IH_y*I_y
)
# Im -> Hm
chm = NaturalConversion(
    subject=c['I_m'],
    outcome=c['Cumulative_hosp'],
    rate_law=r_IH_m*I_m
)
# Io -> Ho
cho = NaturalConversion(
    subject=c['I_o'],
    outcome=c['Cumulative_hosp'],
    rate_law=r_IH_o*I_o
)

### (7) Define observables (including for cumulative cases, hospitalizations, and deaths)

In [15]:
observables_seir = {
    'susceptible': Observable(name='susceptible', expression=S_y+S_m+S_o),
    'exposed': Observable(name='exposed', expression=E_y+E_m+E_o),
    'infected': Observable(name='infected', expression=I_y+I_m+I_o),
    'recovered': Observable(name='recovered', expression=R_y+R_m+R_o),
    'hospita': Observable(name='recovered', expression=R_y+R_m+R_o),
    'recovered': Observable(name='recovered', expression=R_y+R_m+R_o)
}

NameError: name 'Cumulative_hosp' is not defined

### (8) Define template model and save as a petrinet AMR

In [14]:
seir_model = TemplateModel(
    templates=[
        syeyiy,
        syeyim,
        syeyio,
        smemiy,
        smemim,
        smemio,
        soeoiy,
        soeoim,
        soeoio,
        eyiy,
        emim,
        eoio,
        iyry,
        imrm,
        ioro,
    ],
    parameters=parameters,
    initials=initials,
    time=Time(name='t', units=day_units()),
    observables=observables_seir,
    annotations=Annotations(name='SEIRHD age-structured model')
)

# Save as JSON
with open("SEIRHD_age_structured_petrinet.json", 'w') as fh:
    json.dump(template_model_to_petrinet_json(seir_model), fh, indent=1)

# Model 2: SEIRHD model with two vaccine types

# Model 3: SEIRHD model with masking